In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.patches
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import pandas as pd
import flopy

In [ ]:
# make modflow model
exe_name = "/Users/langevin/langevin/dev/modflow6-fork.git/bin/mf6"
sim_ws = Path(f"./mf6_axi")
name = "axi"

Lx = 162000
Ly = 162000
nrow = 151
ncol = 151
dx = Lx / nrow

x = np.linspace(0 + dx / 2., Lx - dx / 2., ncol)
y = np.linspace(Ly - dx / 2, dx / 2, nrow)
X, Y = np.meshgrid(x, y)

rmax = 32188.
r = np.sqrt((X - Lx/2) ** 2 + (Y - Ly / 2) ** 2)
strt = 0.4575 + 0.1525 * np.cos(np.pi * r / rmax)
strt[r > rmax] = 0.305

land_surface = 0.
roughness = 1.0

nper = 1
perlen = nper * [1036800.0]
nstp = nper * [288]
tsmult = nper * [1]

tdis_rc = []
for i in range(nper):
    tdis_rc.append((perlen[i], nstp[i], tsmult[i]))

name = "swf"

# build MODFLOW 6 files
sim = flopy.mf6.MFSimulation(
    sim_name=f"{name}_sim",
    version="mf6",
    exe_name=exe_name,
    sim_ws=sim_ws,
)

# create tdis package
ats_filerecord = None
tdis = flopy.mf6.ModflowTdis(
    sim,
    ats_filerecord=ats_filerecord,
    time_units="SECONDS",
    nper=nper,
    perioddata=tdis_rc,
)

if False:
    # setup ats
    dt0 = 100.  # seconds
    dtmin = 5
    dtmax = 10000.0  # seconds
    dtadj = 2.0
    dtfailadj = 5.0

    ats_filerecord = name + ".ats"
    atsperiod = [
        (0, dt0, dtmin, dtmax, dtadj, dtfailadj),
        (1, dt0, dtmin, dtmax, dtadj, dtfailadj),
    ]
    tdis.ats.initialize(
        maxats=len(atsperiod),
        perioddata=atsperiod,
        filename=ats_filerecord,
    )

def add_model(sim, modelname, dev_swr_conductance):

    # surface water model
    swf = flopy.mf6.ModflowOlf(sim, modelname=modelname, save_flows=True)

    nouter, ninner = 200, 30
    hclose, rclose, relax = 1e-8, 1e-8, 0.97
    imsswf = flopy.mf6.ModflowIms(
        sim,
        print_option="SUMMARY",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="DBD",
        under_relaxation_theta=0.9,
        under_relaxation_kappa=0.0001,
        under_relaxation_gamma=0.0,
        inner_maximum=ninner,
        inner_dvclose=hclose,
        linear_acceleration="BICGSTAB",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        preconditioner_levels=7,
        # backtracking_number=5,
        # backtracking_tolerance=1.0,
        # backtracking_reduction_factor=0.3,
        # backtracking_residual_limit=100.0,
        filename=f"{modelname}.ims",
    )
    sim.register_ims_package(imsswf, [swf.name])

    dis = flopy.mf6.ModflowOlfdis2D(
        swf,
        nrow=nrow,
        ncol=ncol,
        delr=dx,
        delc=dx,
        botm=land_surface,
        xorigin=0.,
    )

    dfw = flopy.mf6.ModflowOlfdfw(
        swf,
        dev_swr_conductance=dev_swr_conductance,
        print_flows=False,
        save_flows=True,
        manningsn=roughness,
        idcxs=None,
    )

    sto = flopy.mf6.ModflowOlfsto(
        swf,
        save_flows=True,
        steady_state={0: False, 1: False},
        transient={0: True, 1: True},
    )

    ic = flopy.mf6.ModflowOlfic(
        swf,
        strt=strt,
    )

    # output control
    oc = flopy.mf6.ModflowOlfoc(
        swf,
        budget_filerecord=f"{modelname}.bud",
        stage_filerecord=f"{modelname}.stage",
        saverecord=[
            ("STAGE", "ALL"),
            ("BUDGET", "ALL"),
        ],
        printrecord=[
            ("BUDGET", "ALL"),
        ],
    )

    # chd
    ibd = np.ones((nrow, ncol), dtype=int)
    ibd[1:-1, 1:-1] = 0
    rows, cols = np.where(ibd == 1)
    spd = [(i, j, 0.305) for i, j in zip(rows, cols)]
    chd = flopy.mf6.ModflowOlfchd(
        swf,
        maxbound=len(spd),
        print_input=True,
        print_flows=True,
        stress_period_data=spd,
    )

swfname = "swf_model"
add_model(sim, swfname, dev_swr_conductance=False)

swrname = "swr_model"
add_model(sim, swrname, dev_swr_conductance=True)

# write the model files
sim.write_simulation()

# run the model
sim.run_simulation()

In [ ]:
def make_plot(modelname, fname=None, include_swr=False):
    modelgrid = flopy.discretization.StructuredGrid(
        nrow=nrow, 
        ncol=ncol,
        delr=np.array(ncol*[dx]),
        delc=np.array(nrow*[dx]),
    )

    times_obs = [50., 100., 150., 200., 250., 288.]
    fpth = sim_ws / f"{modelname}.stage"
    stage_obj = flopy.utils.HeadFile(fpth, text="STAGE")

    if include_swr:
        fpth = Path("./swr-model/WTDecay.stg")
        swr_stage_obj = flopy.utils.SwrStage(fpth)

    fig = plt.figure(figsize=(8, 4))
    ax = fig.add_subplot(1, 2, 1)
    pmv = flopy.plot.PlotMapView(modelgrid=modelgrid) #model=swf)

    contour_line_widths = 0.5
    pmv.contour_array(strt, levels=[0.31], colors="black", linewidths=contour_line_widths)
    ca_colors = ["blue", "cyan", "green", "yellow", "orange", "red"]
    for i, t in enumerate(times_obs):
        stage = stage_obj.get_data(idx=int(t)-1).flatten()
        pmv.contour_array(stage, levels=[0.31], colors=ca_colors[i], linewidths=contour_line_widths)

        if include_swr:
            stage = swr_stage_obj.get_data(totim=t*60.*60.)["stage"]
            pmv.contour_array(stage, levels=[0.31], colors="k", linewidths=contour_line_widths, linestyles="--")

    ax.plot(Lx / 2., Ly / 2, marker="o", mfc="k", mec="k", markersize=2, lw=0., label="domain center")

    obs_radius = [0., 11801.9, 31114.1]
    obs_colors = ["blue", "red", "black"]
    for i, r in enumerate(obs_radius):
        ax.plot(Lx / 2. + r, Ly / 2, marker="o", mfc="none", mew=0.5,
                mec=obs_colors[i], markersize=5, lw=0., label=f"Obs {i}")

    ax.set_xlim(0, Lx)
    ax.set_ylim(0.3, Ly)
    plt.legend(loc="upper right", fontsize=7)
    plt.xlabel("X location, in meters")
    plt.ylabel("Y location, in meters")

    ax = fig.add_subplot(1, 2, 2)
    for ii, r in enumerate(obs_radius):
        i, j = modelgrid.intersect(Lx / 2 + r, Lx / 2.)
        ts = stage_obj.get_ts((0, i, j))
        ax.plot(
            ts[:, 0] / 60. / 60., 
            ts[:, 1], 
            lw=contour_line_widths, 
            color=obs_colors[ii], 
            label=f"MF6 Obs {ii}")

        if include_swr:
            irec = i * nrow + j
            ts = swr_stage_obj.get_ts(irec=irec)
            ax.plot(ts["totim"] / 60. / 60., ts["stage"], 
                    lw=contour_line_widths,
                    ls="--", 
                    color=obs_colors[ii], 
                    label=f"SWR Obs {ii}")

    ax.set_xlim(0, 288)
    ax.set_ylim(0.3, 0.65)
    plt.xlabel("Time, in hours")
    plt.ylabel("Stage, in meters")
    plt.legend(loc="upper right", fontsize=7)
    plt.plot()
    fig.tight_layout()

    if fname is not None:
        plt.savefig(fname, dpi=300)


In [ ]:
from flopy.plot import styles
modelname = swfname
fname = Path("../../doc/figures/") / "axi-results.png"
with styles.USGSMap():
    make_plot(modelname, fname=fname, include_swr=True)

In [ ]:
modelname = swrname
with styles.USGSMap():
    make_plot(modelname, include_swr=True)